<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.981 · TFM · Aula 1</p>
<p style="margin: 0; text-align:right;">2024-2 · Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'informàtica, multimèdia i telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>
<div style="text-align: center; margin-top: 40px;">
    <h1>Aplicación para ayuda a la conducción con deep learning</h1>
</div>

### ARCHIVO 6: Descarga y adecuación de dataset de semáforos y STOPS con COCO

Breve código que descarga únicamente las imágenes de semáforos y STOPS. Después, se adecua los archivos de etiquetas en formato del modelo que va a entrenar: YOLOv11n

Primeramente se accede al directorio de COCO disponible en: https://cocodataset.org/#download [1]

Se debe descargar las anotaciones 2017 Train/Val annotations [241MB]

Se guardan los datos en: /coco/annotations


---
<small>

<a name="note_paper"></a>[1] Tarea no automatizada

</small>

In [ ]:
# Librerias

import os
import json
import requests
from tqdm import tqdm
from pycocotools.coco import COCO

In [ ]:
# Direccions
annotations_path = "coco/annotations/instances_train2017.json" # Arxiu d'anotacions de COCO
output_dir = "coco_filtered/images/"                           # Directiu on es guarden les imatges
coco_url = "http://images.cocodataset.org/train2017/"          # Versió de dades de COCO descarregada

# Dades COCO
coco = COCO(annotations_path)
target_categories = [10, 13] # Selecció únicament de (semàfor: 10, stop: 13)

# Es crea llistat d'ID amb les imatges filtrades a descarregar
image_ids = set()

for cat_id in target_categories:
    image_ids.update(coco.getImgIds(catIds=[cat_id]))

images_info = coco.loadImgs(list(image_ids))

# Descarregar imatges
for img_info in tqdm(images_info, desc="Descarregant imatges"):
    img_url = coco_url + img_info["file_name"]
    img_path = os.path.join(output_dir, img_info["file_name"])

    # Descarregar només si no existeix
    if not os.path.exists(img_path):
        response = requests.get(img_url, stream=True)
        if response.status_code == 200:
            with open(img_path, "wb") as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)

print(f"Descarregades {len(images_info)} imatges de semàfors i STOPS a {output_dir}.")

loading annotations into memory...
Done (t=18.12s)
creating index...
index created!


Descarregant imatges: 100%|██████████| 5705/5705 [1:03:48<00:00,  1.49it/s]

Descarregades 5705 imatges de semàfors i STOPS a coco_filtered/images/.


In [ ]:
# Adequació etiquetes
labels_path = "coco_filtered/labels/"

# Dades COCO
coco = COCO(annotations_path)
target_categories = {3: 0, 4: 1}  # ID per YOLO (0 = Semàfor, 1 = STOP)

image_filenames = set(os.listdir(output_dir)) # S'agafa únicament les etiquetes de les imatges descarregades
image_ids = {img["id"]: img["file_name"] for img in coco.loadImgs(coco.getImgIds()) if img["file_name"] in image_filenames} # Diccionari bucle

# Funció per convertir BBox COCO a YOLO
def coco_to_yolo_bbox(bbox, img_w, img_h):
    '''
    Funció que canvia el format etiqueta COCO per YOLO.

    COCO:
    bbox són les dades de l'objecte, la 'caixa'.
    referent:   x coordenada superior esquerra de la bbox
                y coordenada superior esquerra de la bbox
                w amplada de la bbox
                h alçada de la bbox
    img_w i img_h amplada i alçada de la imatge.

    YOLO:
    Necessita punt central en X i Y i després amplada i alçada de la bbox. s'ha de normalitzar entre 0 i 1.

    Entrada: 
        - bbox: posicio_inicial_x, posicio_inicial_y, amplada, alçada
        - img_w: amplada imatge
        - img_h: alçada imatge

    Sortida:
        - posicio_central_x posicio_central_y amplada alçada
    '''

    # Variables
    x, y, w, h = bbox

    # Càlcul posicions relatives
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w = w / img_w
    h = h / img_h

    return f"{x_center} {y_center} {w} {h}"

# Normalització anotacions
for img_id, img_filename in image_ids.items():

    img_info = coco.loadImgs(img_id)[0] # Carrega imatge
    img_w, img_h = img_info["width"], img_info["height"] # Alçada i Amplada

    ann_ids = coco.getAnnIds(imgIds=[img_id], catIds=list(target_categories.keys())) # Obtè anoracions cotxes i moto
    anns = coco.loadAnns(ann_ids) #obtè les dades

    # Crear label
    label_filename = img_filename.replace(".jpg", ".txt")
    label_path = os.path.join(labels_path, label_filename)

    # Es posen les dades
    with open(label_path, "w") as f:
        for ann in anns:
            category_id = target_categories[ann["category_id"]] # Index nou
            bbox = coco_to_yolo_bbox(ann["bbox"], img_w, img_h)
            f.write(f"{category_id} {bbox}\n")

loading annotations into memory...
Done (t=17.69s)
creating index...
index created!
Conversió completada! Anotacions guardades a coco_filtered/labels/
